In [1]:
import numpy as np
import random
import math
import cmath

In [2]:
def get_distance2D(p1, p2):
    return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

def get_distance3D(p1, p2):
 
 return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2 + (p1[2] - p2[2]) ** 2)

#### Ground to Air Channel Gain

def get_PL_A2A(p1, p2):
 
 d_3D = get_distance3D(p1, p2)
 
 return 20 * math.log(d_3D) + 20 * math.log10(f_c) + 20 * math.log10((4*math.pi) / (3*10**8))

def get_PL_A2G(p1, p2):
 
#  d_3D = get_distance3D(p1, p2)
 eta_LOS, eta_NLOS = 2.0, 21.0 # https://ieeexplore.ieee.org/document/9844892
 PL_A2G_LOS = get_PL_A2A(p1, p2) + eta_LOS
 PL_A2G_NLOS = get_PL_A2A(p1, p2) + eta_NLOS

 return PL_A2G_LOS, PL_A2G_NLOS

def get_Pr_A2G(p1, p2):
 
 a = 0.136 # https://ieeexplore.ieee.org/document/8796414
 b = 11.95 # https://ieeexplore.ieee.org/document/9844892
 d_3D = get_distance3D(p1, p2)
 elevation_angle = math.asin(p2[2]/d_3D)

 return 1 / (1 + a* math.exp(-1 * b * (elevation_angle - a)))

def get_channel_gain_A2G(p1, p2):
 
 PL_A2G_LOS, PL_A2G_NLOS = get_PL_A2G(p1, p2)
 Pr_A2G = get_Pr_A2G(p1, p2)
 PL_A2G_dB = (Pr_A2G * PL_A2G_LOS) + ((1 - Pr_A2G) * PL_A2G_NLOS)
 PL_A2G = 10**(PL_A2G_dB / 10)

 return 1 / PL_A2G

#### Ground to Ground Channel Gain

def get_breakpoint(p1, p2):
    return 4 * (p1[2] - 1) * (p2[2] - 1) * f_c / (3*10**8)

def get_Pr_G2G(p1, p2):
    
    d_2D = get_distance2D(p1, p2)
    
    if d_2D < 18 or d_2D == 18:
        pr_LOS = 1
    else:
        pr_LOS = ((18/d_2D) + (math.exp(-d_2D/63)*(1-(18/d_2D))))
        
    return pr_LOS 

def get_PL_LOS_G2G(p1, p2):
    
    d_2D = get_distance2D(p1, p2)
    BP = get_breakpoint(p1, p2)
    
    if  (10 < d_2D) and (d_2D < BP):
        PL_LOS = 28 + 22*math.log10(get_distance3D(p1, p2)) + 20*math.log10(f_c*10**(-9))
    else:
        PL_LOS = 28 + 40*math.log10(get_distance3D(p1, p2)) + 20*math.log10((f_c*10**(-9))) - 9*math.log10(BP**2 + (p2[2] - p1[2])**2)
    
    return PL_LOS
def get_PL_NLOS_G2G(p1, p2):
    
    PL_NLOS = max(get_PL_LOS_G2G(p1, p2), (13.54 + 39.08 * math.log10(get_distance3D(p1, p2)) + 20 * math.log10(f_c*10**(-9)) - 0.6*(p1[2] -1.5)))
    
    return PL_NLOS

def get_PL_G2G(p1, p2):
    
    return (get_Pr_G2G(p1, p2) * get_PL_LOS_G2G(p1, p2)) + ((1-get_Pr_G2G(p1, p2)) * get_PL_NLOS_G2G(p1, p2))

def get_channel_gain_G2G(p1, p2):
    
    return 1 / 10**(get_PL_G2G(p1, p2)/10)

In [3]:
# Genrating increasing random numbers
def generate_increasing_random_numbers(n, start, end):
    # Generate n random numbers in the range and sort them
    random_numbers = sorted(random.sample(range(start, end), n))
    return random_numbers

In [4]:
#### Topology
f_c = 5*10**9
P_max = 0.1
I0 = 3.98*10**(-21)
W = 5 * 10 ** 6
responders = 12
x_f_p = np.linspace(0.1, 0.3, responders)
responder_height = 1.5
uavs = 3
alpha = 0.2
beta = 0.2
loss_aversion = 5
base_station_coordinates = [-100, -300, 25]
uav_coordinates = [[30, 0, 50], [40, 0, 50], [50, 0, 50]]
uav_coordinates.sort(reverse= True)
maximum_tolerable_interference_uav = [1*(10**(-20)), 2*(10**(-20)), 3*(10**(-20))]
data_per_responder = generate_increasing_random_numbers(responders, 1000, 2000)
w1 = 1
w2 = 100
#2D-List of responders having y, z coordinates 
responder_coordinates = [[0,responder_height] for x in range(responders)]
x_f_D = np.linspace(0.1, 0.3, responders)
responders_per_uav = [[] for x in range(uavs)]
satisfaction_per_uav = dict()
for uav in range(uavs):
    satisfaction_per_uav[uav] = 0
satisfaction_per_uav



responder_uav_distance = list()
responder_bs_distance = list()
responder_uav_channel_gain = list()
responder_bs_channel_gain = list()

#Adding x coordinates to 2D-List of responders to make it 3-D coordinate list
for responder in range(responders):
    responder_coordinates[responder].insert(0,60+(responder+1)+((responder+1)*20))

for responder in range(responders):
    # Temporary List for Each responder which is a row matrix with 3 entries as 3 UAVs.
    responder_uav_distance_temp = list()
    responder_uav_channel_gain_temp = list()
    for uav in range(uavs):
        responder_uav_distance_temp.append(get_distance3D(responder_coordinates[responder], uav_coordinates[uav]))
        responder_uav_channel_gain_temp.append(get_channel_gain_A2G(responder_coordinates[responder], uav_coordinates[uav]))
    responder_uav_distance.append(responder_uav_distance_temp)
    responder_uav_channel_gain.append(responder_uav_channel_gain_temp)
print(responder_uav_channel_gain)

for responder in range(responders):
    responder_bs_distance.append(get_distance3D(responder_coordinates[responder], base_station_coordinates))
    responder_bs_channel_gain.append(get_channel_gain_G2G(responder_coordinates[responder], base_station_coordinates))
print(responder_bs_channel_gain)

[[1.1276994895251199e-13, 7.170163416155519e-14, 4.466624572720398e-14], [4.259755429615106e-14, 2.665824702130775e-14, 1.6964720209204454e-14], [1.6233724355916344e-14, 1.0581562183822434e-14, 7.0577013431733484e-15], [6.786026554860494e-15, 4.63804587004253e-15, 3.235984867511375e-15], [3.1249052172747887e-15, 2.2257535939761907e-15, 1.612449495662738e-15], [1.562662766556337e-15, 1.151305272563006e-15, 8.600638049412806e-16], [8.359389396305814e-16, 6.331751938720362e-16, 4.85129947655803e-16], [4.726612974903606e-16, 3.663655881997886e-16, 2.8674441360324297e-16], [2.7994524325811023e-16, 2.2128337053993957e-16, 1.763827574696217e-16], [1.725033402376578e-16, 1.3868658018595457e-16, 1.123172904024172e-16], [1.1001584064641915e-16, 8.977345380198883e-17, 7.373068093338406e-17], [7.231805650387394e-17, 5.979444852903736e-17, 4.972542996721665e-17]]
[2.883224263409038e-13, 2.5183682708854426e-13, 2.1892340537428807e-13, 1.8969820274587202e-13, 1.6405670188325041e-13, 1.417584508347982

In [5]:
#Eq 1
def get_data_rate_res_uav(responder_id, uav_id):
    # Interference List
    responder_to_uav_channel_gain_power = list()
    
    # Compute the interference of other responders assigned to the same UAV
    for responder in responders_per_uav[uav_id]:
        if responder == responder_id:
            continue
        else:
            responder_to_uav_channel_gain_power.append(get_channel_gain_A2G(responder_coordinates[responder], uav_coordinates[uav_id]) *x_f_p[responder]*P_max)
    
    # Compute the interference of responders assigned to other UAVs
    responder_to_other_uav_channel_gain_power = list()
    for uav in range(uavs):
        if uav == uav_id:
            continue
        else:
            for responder in responders_per_uav[uav]:
                responder_to_other_uav_channel_gain_power.append(get_channel_gain_A2G(responder_coordinates[responder], uav_coordinates[uav]) *x_f_p[responder]*P_max)
    
    interference = sum(responder_to_other_uav_channel_gain_power) + sum(responder_to_uav_channel_gain_power)
    
    data_rate = W * math.log2(1 + ((x_f_p[responder_id] * P_max * get_channel_gain_A2G(responder_coordinates[responder_id], uav_coordinates[uav_id]))/(I0 + interference)))
    return data_rate

#Eq 2
def get_data_rate_res_bs(responder_id):

    responder_to_bs_channel_gain_power = list()
    for responder in range(responders):
        if responder == responder_id:
            continue
        else:
            responder_to_bs_channel_gain_power.append(get_channel_gain_G2G(responder_coordinates[responder], base_station_coordinates) *(1 - x_f_p[responder])*P_max)
    data_rate = W * math.log2(1 + ((x_f_p[responder] * P_max * get_channel_gain_G2G(responder_coordinates[responder_id], base_station_coordinates))/(I0 + sum(responder_to_bs_channel_gain_power))))
    return data_rate

#Eq3
def get_latency_res_uav(responder_id, uav_id):
    # Convert the data transmitted to bits.
    responder_to_uav_latency = (x_f_D[responder_id]*data_per_responder[responder_id])/get_data_rate_res_uav(responder_id, uav_id)
    return responder_to_uav_latency

#Eq4 
def get_latency_res_bs(responder_id):
    responder_to_bs_latency = ((1-x_f_D[responder_id])*data_per_responder[responder_id])/get_data_rate_res_bs(responder_id)
    return responder_to_bs_latency

#Eq5
def get_total_latency_res(responder_id, uav_id):
    total_latency_res =  get_latency_res_uav(responder_id, uav_id) + get_latency_res_bs(responder_id)
    return total_latency_res

#Eq6
def get_energy_res_uav(responder_id, uav_id):
    responder_to_uav_energy = get_latency_res_uav(responder_id, uav_id)*(x_f_p[responder_id]*P_max)
    return responder_to_uav_energy

#Eq7
def get_energy_res_bs(responder_id):
    responder_to_bs_energy = get_latency_res_bs(responder_id)*((1 - x_f_p[responder_id])*P_max)
    return responder_to_bs_energy

#Eq8
def get_total_energy_res(responder_id, uav_id):
    total_energy_res = get_energy_res_uav(responder_id, uav_id) + get_energy_res_bs(responder_id)
    return total_energy_res
#Eq9
def get_total_overhead_res_to_uav(responder_id, uav_id):
    total_overhead = w1*get_total_latency_res(responder_id, uav_id) +w2*get_total_energy_res(responder_id, uav_id)
    return total_overhead

#Eq10
def get_prob_failure_uav(responder_id, uav_id):
    responder_to_uav_channel_gain_power = list()
    
    # Compute interference with responders assigned to same UAV
    for responder in responders_per_uav[uav_id]:
        if responder == responder_id:
            continue
        else:
            responder_to_uav_channel_gain_power.append(get_channel_gain_A2G(responder_coordinates[responder], uav_coordinates[uav_id]) * x_f_p[responder] * P_max)
    
    interference = sum(responder_to_uav_channel_gain_power)
    
    if interference == 0:
        PoF_res_uav = 0
    elif interference < maximum_tolerable_interference_uav[uav_id]:
        PoF_res_uav = maximum_tolerable_interference_uav[uav_id]/interference
    elif interference > maximum_tolerable_interference_uav[uav_id] or interference == maximum_tolerable_interference_uav[uav_id]:
        PoF_res_uav = 1
        
    return PoF_res_uav

#Eq11
def get_expected_overhead_res_to_uav(responder_id, uav_id):
    uav_failed_overhead = (w1*get_latency_res_uav(responder_id, uav_id)) + (w2*get_energy_res_uav(responder_id, uav_id))
    sending_to_bs = (w1*((data_per_responder[responder_id])/get_data_rate_res_bs(responder_id))) + (w2*(((data_per_responder[responder_id])/get_data_rate_res_bs(responder_id))*((1-x_f_p[responder_id])*P_max)))
    first_term = get_prob_failure_uav(responder_id, uav_id)*(uav_failed_overhead + sending_to_bs)
    second_term = (1-get_prob_failure_uav(responder_id, uav_id))*(get_total_overhead_res_to_uav(responder_id, uav_id))
    overhead = first_term + second_term
    return overhead

#Eq12
def get_the_reference_point_res_to_uav(responder_id, uav_id):
    term1 = (((x_f_D[responder]*data_per_responder[responder_id]))/get_data_rate_res_bs(responder_id))
    term2 = term1 * ((1-x_f_p[responder_id])*P_max)
    reference_point = w1*term1 + w2*term2
    return reference_point

def get_the_uav_overhead(responder_id, uav_id):
    overhead = w1*get_latency_res_uav(responder_id, uav_id) + w2*get_energy_res_uav(responder_id, uav_id)
    return overhead

#Eq13/16
def get_prospect_theoretic_utility_res_to_uav(responder_id, uav_id):
    if get_the_uav_overhead(responder_id, uav_id) < get_the_reference_point_res_to_uav(responder_id, uav_id):
        prospect_thoeretic_utility = (get_the_reference_point_res_to_uav(responder_id, uav_id) - get_the_uav_overhead(responder_id, uav_id))**alpha
    else:
        prospect_thoeretic_utility = -loss_aversion *((get_the_uav_overhead(responder_id, uav_id) - get_the_reference_point_res_to_uav(responder_id, uav_id))**beta)
    return prospect_thoeretic_utility

#Eq17
def get_prospect_theoretic_expected_utility_res_to_uav(responder_id, uav_id):
    expected_utility = (1-get_prob_failure_uav(responder_id, uav_id))* (get_the_reference_point_res_to_uav(responder_id, uav_id) - get_the_uav_overhead(responder_id, uav_id))**alpha - (get_prob_failure_uav(responder_id, uav_id)* get_the_uav_overhead(responder_id, uav_id))**beta
    return expected_utility

#Eq18
def get_satisfaction_utility_res_to_uav(responder_id, uav_id):
    print(responder_id, (w1 * (1 - x_f_D[responder_id]) * data_per_responder[responder_id]/ get_data_rate_res_bs(responder_id)), (w2* (1 - x_f_D[responder_id]) * data_per_responder[responder_id] * P_max/ get_data_rate_res_bs(responder_id)))
    satisfaction = get_prospect_theoretic_expected_utility_res_to_uav(responder_id, uav_id) - (w1 * (1 - x_f_D[responder_id]) * data_per_responder[responder_id]/ get_data_rate_res_bs(responder_id)) - (w2* (1 - x_f_D[responder_id]) * data_per_responder[responder_id] * P_max/ get_data_rate_res_bs(responder_id))
    return satisfaction

In [6]:
# Get the min latency profile corresponding to a responder.

def get_latency_profile_res(responder_id):
    latency_profile = list()
    for uav in range(uavs):
        latency_profile.append(get_total_latency_res(responder_id, uav))
    return latency_profile

# Get the min energy corresponding to a responder.
def get_energy_profile_res(responder_id):
    energy_profile = list()
    for uav in range(uavs):
        energy_profile.append(get_total_energy_res(responder_id, uav))
    return energy_profile

global_latency_profile = list()
global_energy_profile = list()

for responder in range(responders):
    global_latency_profile.append(get_latency_profile_res(responder))
    global_energy_profile.append(get_energy_profile_res(responder))
print(global_latency_profile)
print(global_energy_profile)

[[0.0018865939837639178, 0.0018866367495821601, 0.0018866850073661525], [0.0024238614255811704, 0.0024239284160226873, 0.0024239985068932415], [0.002857896948644969, 0.0028579804554682872, 0.0028580659979006587], [0.003346696331771648, 0.0033467948302183873, 0.003346895300551113], [0.0040303957775923698, 0.004030513299752592, 0.0040306332837670728], [0.0046576108654770975, 0.0046577447191008398, 0.0046578816608844973], [0.0056649098269184262, 0.0056650700704060546, 0.0056652343337304147], [0.0066156626422952253, 0.0066158468319206737, 0.0066160359383469645], [0.0078749499011677299, 0.0078751657935736703, 0.0078753877153118716], [0.0096041532273951694, 0.0096044126872266086, 0.0096046796433397336], [0.01292616607491763, 0.012926510546760897, 0.01292686524444467], [0.015151334747116944, 0.015151733583645768, 0.015152144536556047]]
[[0.00016970202839186545, 0.00016970245605004789, 0.0001697029386278878], [0.0002136173985649421, 0.00021361819027016003, 0.00021361901861681202], [0.000246669

In [7]:
for responder_id in range(responders):
    for uav_id in range(uavs):
        print(responder_id, uav_id, get_prospect_theoretic_expected_utility_res_to_uav(responder_id, uav_id))

0 0 0.36276652786
0 1 0.362765540103
0 2 0.362764425485
1 0 0.38155807864
1 1 0.381556699457
1 2 0.381555256421
2 0 0.394497023109
2 1 0.394495393208
2 2 0.394493723547
3 0 0.407325638779
3 1 0.407323817142
3 2 0.407321959005
4 0 0.422942574849
4 1 0.422940571499
4 2 0.422938526144
5 0 0.435559607903
5 1 0.435557444032
5 2 0.435555230194
6 0 0.453175832285
6 1 0.453173483549
6 2 0.453171075843
7 0 0.467699071105
7 1 0.467696551435
7 2 0.467693964448
8 0 0.484547214197
8 1 0.484544508254
8 2 0.484541726677
9 0 0.504457572236
9 1 0.504454658342
9 2 0.504451660189
10 0 0.535655186861
10 1 0.535651991615
10 2 0.535648701437
11 0 0.553290352798
11 1 0.553286948106
11 2 0.553283439894


In [8]:
for responder_id in range(responders):
    for uav_id in range(uavs):
        print(responder_id, uav_id, get_satisfaction_utility_res_to_uav(responder_id, uav_id))

0 0.00188545110693 0.0188545110693
0 0 0.342026565684
0 0.00188545110693 0.0188545110693
0 1 0.342025577926
0 0.00188545110693 0.0188545110693
0 2 0.342024463309
1 0.00242224928439 0.0242224928439
1 0 0.354913336511
1 0.00242224928439 0.0242224928439
1 1 0.354911957328
1 0.00242224928439 0.0242224928439
1 2 0.354910514293
2 0.00285584846865 0.0285584846865
2 0 0.363082689954
2 0.00285584846865 0.0285584846865
2 1 0.363081060052
2 0.00285584846865 0.0285584846865
2 2 0.363079390392
3 0.00334416014243 0.0334416014243
3 0 0.370539877212
3 0.00334416014243 0.0334416014243
3 1 0.370538055576
3 0.00334416014243 0.0334416014243
3 2 0.370536197438
4 0.00402721738689 0.0402721738689
4 0 0.378643183594
4 0.00402721738689 0.0402721738689
4 1 0.378641180244
4 0.00402721738689 0.0402721738689
4 2 0.378639134888
5 0.0046538348131 0.046538348131
5 0 0.384367424959
5 0.0046538348131 0.046538348131
5 1 0.384365261088
5 0.0046538348131 0.046538348131
5 2 0.38436304725
6 0.00566023067511 0.0566023067511


In [9]:
# Matching Algorithm building blocks

def get_interference_at_this_uav(uav_id):
    # Interference List
    responder_to_uav_channel_gain_power = list()
    # Compute the interference of other responders assigned to the same UAV
    for responder in responders_per_uav[uav_id]:
        if responder == responder_id:
            continue
        else:
            responder_to_uav_channel_gain_power.append(get_channel_gain_A2G(responder_coordinates[responder], uav_coordinates[uav_id]) *x_f_p[responder]*P_max)
    
    # Compute the interference of responders assigned to other UAVs
    responder_to_other_uav_channel_gain_power = list()
    for uav in range(uavs):
        if uav == uav_id:
            continue
        else:
            for responder in responders_per_uav[uav]:
                responder_to_other_uav_channel_gain_power.append(get_channel_gain_A2G(responder_coordinates[responder], uav_coordinates[uav]) *x_f_p[responder]*P_max)
    
    interference = sum(responder_to_other_uav_channel_gain_power) + sum(responder_to_uav_channel_gain_power)
    return interference

def get_max_satisfaction_uav_id(satisfaction_per_uav):
    uav_id = np.argmax(satisfaction_per_uav)
    uav_id_value = satisfaction_per_uav[uav_id]
    return uav_id

def get_assigned_uav_for_a_responder(responder_id):
    for uav_id, responder_list in enumerate(responders_per_uav):
        if responder_id in responder_list:
            return uav_id
    return None  # Return None if the responder is not found in any UAV
#Eq 19
def get_aggregate_satisfaction_of_uav(uav_id):
    total_satisfaction = 0
    responder_list = responders_per_uav[uav_id]
    for responder_id in responder_list:
        total_satisfaction = total_satisfaction + get_satisfaction_utility_res_to_uav(responder_id, uav_id)
    return total_satisfaction

#Step 1: Generate a random agent id
def get_random_agent(agents):
    random_id = random.randint(0, agents)
    return random_id

# Step 2: Check for assignment of the responder
def get_responder_assignment(responder_id):
    for responder_list in responders_per_uav:
        if responder_id in responder_list:
            return uav_id
    return None

#Step 3:Check the cases for the random responder
def unassigned_responder_case(responder_id):
    # Case 1: Where the repsonder is unassigned
        print("Case 1: Unassigned Responder")
        for uav_id in range(uavs):
            temp = get_aggregate_satisfaction_of_uav(uav_id) + get_satisfaction_utility_res_to_uav(responder_id, uav_id)
            if (temp > get_aggregate_satisfaction_of_uav(uav_id)):
                satisfaction_per_uav[uav_id] = temp
            else:
                satisfaction_per_uav[uav_id] = 0
        # Find the UAV that has the maximum satisfaction.        
        best_uav = get_max_satisfaction_uav_id(satisfaction_per_uav)
        # Perform the assignment.
        responders_per_uav[best_uav].append(responder_id)

def assigned_responder_case(responder_id):
        print("Case 2: Already Assigned UAV")
        # Find the assigned UAV.
        uav_id = get_assigned_uav_for_a_responder(responder_id)
        print(uav_id)
       
        # Generate another randome UAV ID.
        next_uav_id = get_random_agent(uavs)
        if next_uav_id == uav_id:
            while(next_uav_id == uav_id):
                next_uav_id = get_random_agent(uavs)
        # temp_uav_list = list()
        # for uav in range(uavs):
        #     if uav != uav_id:
        #         temp_uav_list.append(uav)
        # next_uav_id = np.random.choice(temp_uav_list)

        # Check Interference Levels
        #Case 2(a): where the interference levels are less at next_uav_id[u']
        if (get_interference_at_this_uav(next_uav_id) < maximum_tolerable_interference_uav[next_uav_id]):
            exchanged_utitily_uav_id = get_aggregate_satisfaction_of_uav(uav_id) - get_satisfaction_utility_res_to_uav(responder_id, uav_id)
            exchanged_utitily_next_uav_id = get_aggregate_satisfaction_of_uav(next_uav_id) + get_satisfaction_utility_res_to_uav(responder_id, next_uav_id)
            if((exchanged_utitily_uav_id + exchanged_utitily_next_uav_id) < (get_aggregate_satisfaction_of_uav(uav_id) + get_aggregate_satisfaction_of_uav(next_uav_id))):
                # Perform the exchange
                responders_per_uav[uav_id].remove(responder_id)
                responders_per_uav[next_uav_id].append(responder_id)
        else:
            # Case 2(b): where the interference levels are high at next_uav_id[u']
            # Randomly select any responder assigned to the UAV with next_uav_id
            random_responder_id = random.choice(responders_per_uav[next_uav_id])
            exchange_utility_uav_id = get_aggregate_satisfaction_of_uav(uav_id) - get_satisfaction_utility_res_to_uav(responder_id, uav_id) + get_satisfaction_utility_res_to_uav(random_responder_id, uav_id)
            exchange_utility_next_uav_id = get_aggregate_satisfaction_of_uav(next_uav_id) - get_satisfaction_utility_res_to_uav(random_responder_id, uav_id) + get_satisfaction_utility_res_to_uav(responder_id, uav_id)
            gained_utility = exchange_utility_uav_id + exchange_utility_next_uav_id
            existing_utility = get_aggregate_satisfaction_of_uav(uav_id) + get_aggregate_satisfaction_of_uav(next_uav_id)
            if gained_utility > existing_utility:
                # Perform the exchange
                responders_per_uav[uav_id].remove(responder_id)
                responders_per_uav[next_uav_id].remove(random_responder_id)
                responders_per_uav[uav_id].append(random_responder_id)
                responders_per_uav[next_uav_id].append(responder_id)

In [10]:
get_aggregate_satisfaction_of_uav(2), get_satisfaction_utility_res_to_uav(0, 1), satisfaction_per_uav

0 0.00188545110693 0.0188545110693


(0, 0.34202557792641441, {0: 0, 1: 0, 2: 0})

In [11]:
get_max_satisfaction_uav_id(satisfaction_per_uav), responders_per_uav[0]

(0, [])

In [12]:
# The Matching Algorithm
max_count = 200
convergence = 0 
def matching_algorithm():

    responders_per_uav_record = list()
    for iteration in range(max_count):
        for responder_id in range(responders):
            # Check assignment
            uav_id = get_responder_assignment(responder_id)
            print(uav_id)
            if (uav_id == None):
                unassigned_responder_case(responder_id)
            else:
                assigned_responder_case(responder_id)
        
        responders_per_uav_record.append(responders_per_uav)

        if iteration > 1:
            if responders_per_uav_record[-1] == responders_per_uav_record[-2]:
                convergence == 1
        if convergence == 1:
            break

In [13]:
unassigned_responder_case(0)

Case 1: Unassigned Responder
0 0.00188545110693 0.0188545110693
0 0.00188545110693 0.0188545110693
0 0.00188545110693 0.0188545110693
